In [7]:
%% cd '/content/drive/Othercomputers/Il mio MacBook Air/DataScience/PROFESSION_AI/NLP/progetto_finale'

UsageError: Cell magic `%%cd` not found (But line magic `%cd` exists, did you mean that instead?).
